In [1]:
# tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f'length of text: {len(text)}')

length of text: 1115394


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# tokenize: maps tokens to integers
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for s, i in stoi.items()}
encode = lambda sentence: [stoi[c] for c in sentence]
decode = lambda tokens: ''.join([itos[i] for i in tokens])

print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


## Other popular encoders:
* [SentencePiece](https://github.com/google/sentencepiece) (Google)
* [TikToken](https://github.com/openai/tiktoken) (OpenAI)

These take more bytes per token, so less number of tokens per sentence. They
also have larger vocab size given it's trained over larger datasets.

Side note on installing packages:
* Prefer to install within uv's venv.
* Also need to hook to Jupyter kernel. This needs (a) ctrl+shift+P in VSCode to
  select python kernel to venv, and (b) in upper-right corner, change kernel to
  point to uv's venv.

In [6]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')
enc.n_vocab
print(enc.encode('hello world'))
print(enc.decode([31373, 995]))

[31373, 995]
hello world


In [12]:
import torch
import numpy as np

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

# text is a list of characters (1115394 chars)
# encode(text) is a list of ints (1115394 ints)
# torch.tensor(encode(text)) transforms into a 1-d tensor (shape == (1115394,))
assert data.numpy().shape == (1115394,)

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [15]:
# Train / validation data split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]